In [27]:
import torch

from transformers import AutoModelForCausalLM, pipeline, GPT2LMHeadModel, GPT2Tokenizer

import numpy as np
import sympy as sp

In [2]:
from model.tokens import Token, TOKEN_TYPE_EXPRESSIONS, TOKEN_TYPE_ANSWERS
from model.equation_interpreter import Equation
from model.vocabulary import Vocabulary
from model.tokens import Token

In [3]:
from datasets import disable_caching
disable_caching()

In [4]:
# Create a combined vocabulary
vocabulary = Vocabulary.construct_from_list(TOKEN_TYPE_EXPRESSIONS + TOKEN_TYPE_ANSWERS)
vectorized_sample = vocabulary.vectorize(["#", "/", "0", "-1", "[SEP]", "TT_INTEGER"])
vectorized_sample, [vocabulary.getToken(idx) for idx in vectorized_sample]

# Global variables
model_name = "JustSumAI"
project_name = "JustSumAI"
repo_name = f"{model_name}_cleaned_gpt2_data"

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

input_text = "This is my input sequence."
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids, input_ids.size()

(tensor([[1212,  318,  616, 5128, 8379,   13]]), torch.Size([1, 6]))

In [6]:
vocabulary.end_seq_index

65

# Load model

In [7]:
model = GPT2LMHeadModel.from_pretrained(f"Dragonoverlord3000/{model_name}", force_download=True)
model

C:\Users\Hugo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hugo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(68, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=68, bias=False)
)

In [20]:
test_example_ids = torch.LongTensor([vocabulary.vectorize(["#", "/", "0", "0"])[:-1] + [vocabulary.separator_index]])
test_example_ids[0], test_example_ids.size()

(tensor([64, 40, 39, 19, 19, 67]), torch.Size([1, 6]))

In [21]:
test = model(test_example_ids).logits
print(test.size())
[vocabulary.getToken(torch.argmax(o).item()) for o in test[0]]

torch.Size([1, 6, 68])


['<BEGIN>', '#', '/', '0', '0', 'TT_INTEGER']

In [22]:
out = model.generate(test_example_ids, 
                     eos_token_id=vocabulary.end_seq_index, 
                     pad_token_id=vocabulary.mask_index)
out, out.size()

(tensor([[64, 40, 39, 19, 19, 67, 49, 49, 49, 53, 44, 60, 58, 49, 57, 60, 58, 49,
          61, 65]]),
 torch.Size([1, 20]))

In [23]:
l = [vocabulary.getToken(o.item()) for o in out[0]]
l, l.index("[SEP]")

(['<BEGIN>',
  '#',
  '/',
  '0',
  '0',
  '[SEP]',
  'TT_INTEGER',
  'TT_INTEGER',
  'TT_INTEGER',
  'TT_SQRT',
  'TT_PI',
  'TT_MULTIPLY',
  'TT_PLUS',
  'TT_INTEGER',
  'TT_LOG',
  'TT_MULTIPLY',
  'TT_PLUS',
  'TT_INTEGER',
  'TT_DIVIDE',
  '<END>'],
 5)

In [24]:
eq = Equation([Token(vocabulary.getToken(o.item())) for o in out[0]][l.index("[SEP]")+1:-1], notation="postfix")
eq

In [25]:
eq.getMathmetaicalNotation()

'((Z+((Z+(Sqrt(Z)*Pi))*Log(Z)))/Z)'

______

In [28]:
sp.parse_expr(eq.getMathmetaicalNotation(), evaluate=False)

(Z + (Pi*Sqrt(Z) + Z)*Log(Z))/Z

### Accept user input

In [29]:
from IPython.display import display
import ipywidgets as widgets

In [70]:
math_display = widgets.HTMLMath(
    value=f"Some math and <i>HTML</i>: \(x^2\) and $$2^\pi + {root_selected.value}$$",
    placeholder='',
    description='',
)

a = widgets.SelectionSlider(
    options=TOKEN_TYPE_EXPRESSIONS[:-2],
    value='-5',
    description='Root c',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
b = widgets.SelectionSlider(
    options=TOKEN_TYPE_EXPRESSIONS[:-2],
    value='-5',
    description='Root c',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
c = widgets.SelectionSlider(
    options=TOKEN_TYPE_EXPRESSIONS[:-2],
    value='-5',
    description='Root c',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print(a, b, c)
    sum_str = r"$$\sum_{n=1}^{\infty}\frac{" + f"(n - {a})(n - {b})(n - {c})" + "}{" + f"(n - a)" + "}" + "$$"
    math_display.value = sum_str
    display(math_display)

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()